In [ ]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# long addition

## 10進数の筆算

* $1234+567$ を筆算で計算する:
  - 桁を揃える
  - 下位の桁から加算し、繰り上げを行う

### 桁を揃えて下位の桁から計算する準備をする

* 整数を文字列に変換し逆順に並べ替える: ``str()``, ``reversed()``
  - 一桁の加算を行うため、文字列から整数に戻す: ``map(int, ...)``
* 下位の桁からペアを作り、桁数が揃わない上位の桁は $0$ で埋める: ``zip()``, ``zip_longest(fillvalue=0)``

In [8]:
list(map(int,reversed(str(1234)))), list(map(int,reversed(str(567))))

([4, 3, 2, 1], [7, 6, 5])

In [2]:
from itertools import zip_longest

In [5]:
[(n,m) for n,m in zip_longest(map(int,reversed(str(1234))), map(int,reversed(str(567))), fillvalue=0)]

[(4, 7), (3, 6), (2, 5), (1, 0)]

In [9]:
[divmod(n+m,10) for n,m in zip_longest(map(int,reversed(str(1234))), map(int,reversed(str(567))), fillvalue=0)]

[(1, 1), (0, 9), (0, 7), (0, 1)]

In [30]:
list(zip(*[divmod(n+m,10) for n,m in zip_longest(map(int,reversed(str(1234))), map(int,reversed(str(567))), fillvalue=0)]))

[(1, 0, 0, 0), (1, 9, 7, 1)]

In [18]:
def long_addition(a,b):
    a = map(int,reversed(str(a)))
    b = map(int,reversed(str(b)))
    q=0
    for n,m in zip_longest(a, b, fillvalue=0):
        q, r = divmod(q+n+m,10)
        yield str(r)
    if q:
        yield str(q)

In [23]:
list(long_addition(1234, 567))

['1', '0', '8', '1']

In [24]:
list(reversed(list(long_addition(1234, 567))))

['1', '8', '0', '1']

In [26]:
''.join(reversed(list(long_addition(1234, 567))))

'1801'

In [21]:
int(''.join(reversed(list(long_addition(1234, 567)))))

1801

In [22]:
1234+567

1801

## 2進法の筆算
### Leibnizによる九九のない世界

In [18]:
def long_addition(a,b):
    a = map(int,reversed(str(a)))
    b = map(int,reversed(str(b)))
    q=0
    for n,m in zip_longest(a, b, fillvalue=0):
        q, r = divmod(q+n+m,10)
        yield str(r)
    if q:
        yield str(q)

In [37]:
bin(1234)[2:], bin(567)[2:], bin(1234+567)[2:]

('10011010010', '1000110111', '11100001001')

In [76]:
def long_addition2(a,b):
    a = map(int,reversed(bin(a)[2:]))
    b = map(int,reversed(bin(b)[2:]))
    q=0
    for n,m in zip_longest(a, b, fillvalue=0):
        q, r = divmod(q+n+m,2)
        print("{:d}+{:d}+{:d}={:d}{:d}".format(q,n,m, q,r))
        yield str(r)
    if q:
        yield str(q)

In [77]:
list(long_addition2(1234,567))

0+0+1=01
1+1+1=10
1+0+1=10
0+0+0=01
1+1+1=10
1+0+1=10
1+1+0=10
1+1+0=10
0+0+0=01
0+0+1=01
0+1+0=01


['1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '1']

In [53]:
int(''.join(reversed(list(long_addition2(1234, 567)))))

11100001001

In [54]:
int(''.join(reversed(list(long_addition2(1234, 567)))),2)

1801

### Booleによる四則演算のない世界

In [74]:
def long_addition2(a,b):
    a = map(int,reversed(bin(a)[2:]))
    b = map(int,reversed(bin(b)[2:]))
    q=0
    for n,m in zip_longest(a, b, fillvalue=0):
        #q, r = divmod(q+n+m,2)
        print("{:d}+{:d}+{:d}=({:d}&{:d}|{:d}&{:d}|{:d}&{:d})({:d}|{:d}|{:d})".format(q,n,m, q,n,q,m,n,m, q,n,m), end="")
        r = q ^ n ^ m
        q = q & n | q & m | n & m
        print("={:d}{:d}".format(q,r))
        yield str(r)
    if q:
        yield str(q)

In [75]:
int(''.join(reversed(list(long_addition2(1234, 567)))),2)

0+0+1=(0&0|0&1|0&1)(0|0|1)=01
0+1+1=(0&1|0&1|1&1)(0|1|1)=10
1+0+1=(1&0|1&1|0&1)(1|0|1)=10
1+0+0=(1&0|1&0|0&0)(1|0|0)=01
0+1+1=(0&1|0&1|1&1)(0|1|1)=10
1+0+1=(1&0|1&1|0&1)(1|0|1)=10
1+1+0=(1&1|1&0|1&0)(1|1|0)=10
1+1+0=(1&1|1&0|1&0)(1|1|0)=10
1+0+0=(1&0|1&0|0&0)(1|0|0)=01
0+0+1=(0&0|0&1|0&1)(0|0|1)=01
0+1+0=(0&1|0&0|1&0)(0|1|0)=01


1801